In [27]:
import torch
from torch.utils.data import DataLoader
import pandas as pd
import os
import gc
import importlib
import multiprocessing

import sentiment_utils
importlib.reload(sentiment_utils)
from sentiment_utils import SentimentAnalysisModel, CommentsDataset, process_all_batches

In [28]:
multiprocessing.set_start_method("fork", force=True)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [29]:
output_file_path = '../data/beer_reviews_text.csv'

num_rows= 6000

df = pd.read_csv(output_file_path, nrows=num_rows)
df.head()

,text
0,"Puszka 0,33l dzięki Christoph . Kolor jasnozło..."
1,Cerveza pale lager gabonesa. MÃ¡s floja que la...
2,"Kolor- złoty, klarowny. Piana - drobna, średni..."
3,"Botella, de GabÃ³n regalo familiar.31/01/2015C..."
4,Many thanks for this beer to Erzengel. Pours l...


In [35]:
sentiment_model = SentimentAnalysisModel(model_path="../models/sentiment_model_1")

batch_size = 64
dataset = CommentsDataset(df["text"].astype(str).tolist())
data_loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=12,
    pin_memory=True,
    persistent_workers=True
)

df["sentiment_bert"] = process_all_batches(data_loader, sentiment_model)
df.head()


Processing Batches: 100%|██████████| 94/94 [01:25<00:00,  1.11batch/s]


,text,sentiment_bert
0,"Puszka 0,33l dzięki Christoph . Kolor jasnozło...",0.264435
1,Cerveza pale lager gabonesa. MÃ¡s floja que la...,-0.433727
2,"Kolor- złoty, klarowny. Piana - drobna, średni...",0.146698
3,"Botella, de GabÃ³n regalo familiar.31/01/2015C...",-0.067261
4,Many thanks for this beer to Erzengel. Pours l...,0.842057


In [31]:
torch.mps.empty_cache()
gc.collect()

4035

In [32]:
df

,text,sentiment_bert
0,"Puszka 0,33l dzięki Christoph . Kolor jasnozło...",0.264435
1,Cerveza pale lager gabonesa. MÃ¡s floja que la...,-0.433727
2,"Kolor- złoty, klarowny. Piana - drobna, średni...",0.146698
3,"Botella, de GabÃ³n regalo familiar.31/01/2015C...",-0.067261
4,Many thanks for this beer to Erzengel. Pours l...,0.842057
...,...,...
5995,How: Bottle Apperance: Clear yellow. Aroma: fl...,-0.140594
5996,Typical international lager. Weak aroma with s...,-0.067390
5997,Een pilsbier dat heel goudblond eruitziet als ...,-0.259747
5998,"Clear golden, huge frothy white head. Grainy, ...",0.315399
